In [27]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

import tkinter as tk
from tkinter import filedialog as fd

import collections

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm

In [28]:
# open file dialog / catastro de avance
root = tk.Tk()
root.attributes('-topmost', True)
root.iconify()
file_name = fd.askopenfilename(parent=root, filetypes=[(".shp", ".shp")])
root.destroy()

# open file dialog / xlsx de avance
root = tk.Tk()
root.attributes('-topmost', True)
root.iconify()
file_name = fd.askopenfilename(parent=root, filetypes=[(".xlsx", ".xlsx")])
root.destroy()

In [30]:
file_cat_avance = r'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/REPORTE_AVANCE_COSECHA/SHP/cosecha090623.shp'
file_xlsx_avance = r'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/REPORTE_AVANCE_COSECHA/RENDIMIENTO AVANCE 090623.xlsx'
file_xlsx_estimativa = r'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/REPORTE_AVANCE_COSECHA/ESTIMATIVAS_DB.xlsx'

In [31]:
catastro_avance = gpd.read_file(file_cat_avance)

In [33]:
data_avance = pd.read_excel(file_xlsx_avance)

In [34]:
# extraer codigos de cosecha
codigos_cosecha = list(set(data_avance['COD COS']))

In [35]:
print(codigos_cosecha)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226,

## COMPROMISO Y ENTREGAS

In [144]:
# seleccion de grupo aguilera
cod_co = 30
# filtrado por codigo de cosecha
grupo = data_avance[data_avance['COD COS'] == cod_co]
# seleccion de columnas de interes
grupo = grupo[['CODIGO CAÑERO ', 'COD COS', 'NOMBRE CAÑERO', 'TN COMPROMISO INDIVIDUAL', 'TN ENTREGA INDIVIDUAL']]
# calculo de % de avance de entrega del grupo
grupo['avance_porcen'] = (grupo['TN ENTREGA INDIVIDUAL'] / grupo['TN COMPROMISO INDIVIDUAL'])*100
# cambio del nombre del campo codigo cañero
grupo = grupo.rename(columns={'CODIGO CAÑERO ': 'cod_ca'})
grupo

,cod_ca,COD COS,NOMBRE CAÑERO,TN COMPROMISO INDIVIDUAL,TN ENTREGA INDIVIDUAL,avance_porcen
0,75,30,AGUILERA TARADELLES NELSON MARIANO,26992.97,0.00,0.000000
1,577,30,AGUILERA MARIA INGRID WENDE DE,13433.25,631.64,4.702064
2,794,30,AGUILERA WENDE MARIANO,18084.70,15218.32,84.150249
3,8005,30,AGUILERA WENDE MARIA INGRID,5201.02,20.45,0.393192
4,41715,30,VACA DIEZ UGARTE NICOLAS,1781.38,0.00,0.000000


In [149]:
list_obj_grupo = []
for index, row in grupo.iterrows():
    canero = {
        'cod_ca' : row['cod_ca'],
        'nom_ca' : row['NOMBRE CAÑERO'],
        'tn_compro_indiv' : row['TN COMPROMISO INDIVIDUAL'],
        'tn_entrega_indiv' : row['TN ENTREGA INDIVIDUAL'],
        'avance_porcen' : row['avance_porcen']
    }
    list_obj_grupo.append(collections.namedtuple("canero", canero.keys())(*canero.values()))
list_obj_grupo

[canero(cod_ca=75, nom_ca='AGUILERA TARADELLES NELSON MARIANO', tn_compro_indiv=26992.97, tn_entrega_indiv=0.0, avance_porcen=0.0),
 canero(cod_ca=577, nom_ca='AGUILERA MARIA INGRID WENDE DE', tn_compro_indiv=13433.25, tn_entrega_indiv=631.64, avance_porcen=4.702063908585041),
 canero(cod_ca=794, nom_ca='AGUILERA WENDE MARIANO', tn_compro_indiv=18084.7, tn_entrega_indiv=15218.32, avance_porcen=84.15024855264394),
 canero(cod_ca=8005, nom_ca='AGUILERA WENDE MARIA INGRID', tn_compro_indiv=5201.02, tn_entrega_indiv=20.45, avance_porcen=0.3931921046256311),
 canero(cod_ca=41715, nom_ca='VACA DIEZ UGARTE NICOLAS', tn_compro_indiv=1781.38, tn_entrega_indiv=0.0, avance_porcen=0.0)]

In [150]:
# totales de tabla de entreas del grupo seleccionado
grupo_totales = {
    'total_compromiso':grupo['TN COMPROMISO INDIVIDUAL'].sum(),
    'total_entrega':grupo['TN ENTREGA INDIVIDUAL'].sum(),
    'total_avance_porcen':grupo['TN ENTREGA INDIVIDUAL'].sum() / grupo['TN COMPROMISO INDIVIDUAL'].sum() * 100
}
grupo_totales
obj_grupo_totales = collections.namedtuple("totales", grupo_totales.keys())(*grupo_totales.values())
obj_grupo_totales

totales(total_compromiso=65493.32, total_entrega=15870.41, total_avance_porcen=24.232104892529495)

## AVANCE PROPIEDAD

In [151]:
xlsx_estimativas = pd.read_excel(file_xlsx_estimativa, sheet_name='PRODUCCION_POR_PROP')
xlsx_estimativas = xlsx_estimativas.rename(columns={'CODIGO' : 'unidad_01'})

In [152]:
hoy = datetime.now()
mes_actual = hoy.month
mes_actual

8

In [154]:
# cambio de nombre de columna para hacer merge con la tabla de grupo
catastro_avance = catastro_avance.rename(columns={'unidad_03':'cod_ca'})
# merge 
catastro_merge = pd.merge(grupo, catastro_avance, on='cod_ca')
# crea nueva columna "area_cosecha", se almacena en area si el campo estado="cosechado"
catastro_merge['area_cosechada'] = catastro_merge.apply(lambda row : row['area'] if row['estado'] == 'cosechado' else 0, axis=1)
# calculo de area cosechada en el mes actual
catastro_merge['avance_mes'] = catastro_merge.apply(lambda row : row['area'] if row['fc']!=None and row['fc'].split('-')[1] == str(mes_actual) else 0, axis=1)
# agrupacion de datos por pripiedad, resumiendo por area, area_cosechada, avance_mes
propiedades = catastro_merge.groupby(['unidad_01', 'unidad_02'])[['area', 'area_cosechada', 'avance_mes']].sum()
# reseteo de index
propiedades = propiedades.reset_index()
# calculo de % de avance de cosecha
propiedades['%avance'] = propiedades['area_cosechada'] / propiedades['area'] * 100
# merge con las estimativas de produccion de cada propiedad
propiedades = pd.merge(propiedades, xlsx_estimativas, on='unidad_01')
# seleccion de columnas de interes
propiedad = propiedades[['unidad_01', 'unidad_02', 'area', 'area_cosechada', '%avance', 'PRODUCCION']]
propiedad

,unidad_01,unidad_02,area,area_cosechada,%avance,PRODUCCION
0,216,LA_ESTACADA--AGUILERA,226.34,0.00,0.000000,15288.106548
1,217,FLAMBOYAN--AGUILERA,207.46,0.00,0.000000,14102.152708
2,256,LOS_ANDES--AGUILERA,423.24,153.93,36.369436,33770.169576


In [158]:
list_obj_props = []
for index, row in propiedad.iterrows():
    prop = {
        'cod_prop' : row['unidad_01'],
        'nom_prop' : row['unidad_02'],
        'area' : row['area'],
        'area_cosechada' : row['area_cosechada'],
        'avance_porcen' : row['%avance'],
        'produccion' : row['PRODUCCION']
    }
    list_obj_props.append(collections.namedtuple("propiedad", prop.keys())(*prop.values()))
list_obj_props

[propiedad(cod_prop=216, nom_prop='LA_ESTACADA--AGUILERA', area=226.34, area_cosechada=0.0, avance_porcen=0.0, produccion=15288.10654838978),
 propiedad(cod_prop=217, nom_prop='FLAMBOYAN--AGUILERA', area=207.45999999999998, area_cosechada=0.0, avance_porcen=0.0, produccion=14102.152708359197),
 propiedad(cod_prop=256, nom_prop='LOS_ANDES--AGUILERA', area=423.24, area_cosechada=153.93, avance_porcen=36.3694357811171, produccion=33770.16957626864)]

In [160]:
# totales de tabla por columna
propiedades_totales = {
    'total_area':propiedades['area'].sum(),
    'total_area_cosechada':propiedades['area_cosechada'].sum(),
    'total_avance_mes':propiedades['avance_mes'].sum(),
    'total_avance_porcen':propiedades['area_cosechada'].sum() / propiedades['area'].sum() * 100,
    'total_proudccion' : propiedad['PRODUCCION'].sum()
}
propiedades_totales
obj_props_totales = collections.namedtuple("total", propiedades_totales.keys())(*propiedades_totales.values())
obj_props_totales

total(total_area=857.04, total_area_cosechada=153.93, total_avance_mes=0, total_avance_porcen=17.96065527863344, total_proudccion=63160.42883301762)

# AVANCE POR VARIEDAD

In [138]:
# merge 
catastro_merge_2 = pd.merge(grupo, catastro_avance, on='cod_ca')
# crea nueva columna "area_cosecha", se almacena en area si el campo estado="cosechado"
catastro_merge_2['area_cosechada'] = catastro_merge_2.apply(lambda row : row['area'] if row['estado'] == 'cosechado' else 0, axis=1)
# extraer valores unicos en el campo unidad_02
nom_props = list(set(catastro_merge_2['unidad_02']))
# lista de objetos para almacenar una lista de propiedades
list_obj_props_var = []
for prop in nom_props:
    # dict para almacenar una propiedad en concreto y su avance por variedad
    data_obj_props = {}
    data_obj_props['prop'] = prop
    # selecciona todos los datos de una priedad en concreto segun en nombre en el campo unidad_02
    avance_variedad = catastro_merge_2[catastro_merge_2['unidad_02']==prop]
    # agrupa los datos segun variedad
    avance_variedad = avance_variedad.groupby(['variedad'])[['area_cosechada', 'area']].sum()
    # reinicia el index de dataframe
    avance_variedad = avance_variedad.reset_index()
    # ordemar por area cosechadad
    avance_variedad = avance_variedad.sort_values(['area_cosechada'], ascending=False)
    # calculo de porcentaje de avance
    avance_variedad['porcen_avan'] = avance_variedad.apply(lambda row : row['area_cosechada']/row['area'] *100, axis=1)
    # suma de totales
    totales_variedad = {
        'total_cosechado' : avance_variedad['area_cosechada'].sum(),
        'total_area' : avance_variedad['area'].sum(),
        'total_porcen' : avance_variedad['area_cosechada'].sum() / avance_variedad['area'].sum() * 100
    }
    # add totales a dict de datos de la prop
    data_obj_props['totales'] = collections.namedtuple("totales", totales_variedad.keys())(*totales_variedad.values())
    
    # objeto para almacenar las variedades
    list_obj_vars = []
    # recorres el dataframe de avande por variedad para convertir en obj cada fila
    for index, row in avance_variedad.iterrows():
        var = {
            'var':row['variedad'],
            'area_cosecha':row['area_cosechada'],
            'area':row['area'],
            'porcen_avan':row['porcen_avan']
        }
        list_obj_vars.append(collections.namedtuple("variedad", var.keys())(*var.values()))
    data_obj_props['variedades'] = list_obj_vars
    list_obj_props_var.append(collections.namedtuple("propiedad", data_obj_props.keys())(*data_obj_props.values()))
list_obj_props_var

[propiedad(prop='LA_ESTACADA--AGUILERA', totales=totales(total_cosechado=0.0, total_area=226.34, total_porcen=0.0), variedades=[variedad(var='CITTCA_85-22', area_cosecha=0.0, area=33.11, porcen_avan=0.0), variedad(var='RB-2', area_cosecha=0.0, area=69.25999999999999, porcen_avan=0.0), variedad(var='SP_83-5073', area_cosecha=0.0, area=18.98, porcen_avan=0.0), variedad(var='UCG_90-20', area_cosecha=0.0, area=104.99000000000001, porcen_avan=0.0)]),
 propiedad(prop='FLAMBOYAN--AGUILERA', totales=totales(total_cosechado=0.0, total_area=207.45999999999998, total_porcen=0.0), variedades=[variedad(var='CITTCA_85-22', area_cosecha=0.0, area=46.59, porcen_avan=0.0), variedad(var='RB-2', area_cosecha=0.0, area=35.26, porcen_avan=0.0), variedad(var='RBB_77-26', area_cosecha=0.0, area=2.03, porcen_avan=0.0), variedad(var='SP_83-5073', area_cosecha=0.0, area=40.72, porcen_avan=0.0), variedad(var='UCG_90-20', area_cosecha=0.0, area=82.86, porcen_avan=0.0)]),
 propiedad(prop='LOS_ANDES--AGUILERA', tot

## AVANCE POR CORTE

In [140]:
# merge 
catastro_merge_3 = pd.merge(grupo, catastro_avance, on='cod_ca')
# crea nueva columna "area_cosecha", se almacena en area si el campo estado="cosechado"
catastro_merge_3['area_cosechada'] = catastro_merge_3.apply(lambda row : row['area'] if row['estado'] == 'cosechado' else 0, axis=1)
# extraer valores unicos en el campo unidad_02
nom_props = list(set(catastro_merge_3['unidad_02']))
# lista de objetos para almacenar una lista de propiedades
list_obj_props_soca = []
for prop in nom_props:
    # dict para almacenar una propiedad en concreto y su avance por variedad
    data_obj_props = {}
    data_obj_props['prop'] = prop
    avance_corte = catastro_merge_3[catastro_merge_3['unidad_02']==prop]
    
    suma_area = avance_corte.loc[avance_corte['soca'] >= 10, 'area'].sum()
    suma_area_cosechada = avance_corte.loc[avance_corte['soca'] >= 10, 'area_cosechada'].sum()
    avance_corte= avance_corte[avance_corte['soca'] < 10]
    avance_corte['soca'] = avance_corte['soca'].astype(str)
    new_reg = {'soca': '>10', 'area': suma_area, 'area_cosechada': suma_area_cosechada}
    avance_corte = avance_corte.append(new_reg, ignore_index=True)
    #agrupacion de datos por propiedad, resumiendo por area, area_cosechada, avance_mes
    avance_corte = catastro_merge_3.groupby(['soca'])[['area_cosechada', 'area']].sum()
    # reseteo de index
    avance_corte = avance_corte.reset_index()
    # ordemar por soca
    avance_corte = avance_corte.sort_values(['soca'])
    # calculo de porcentaje de avance
    avance_corte['porcen_avan'] = avance_corte.apply(lambda row : row['area_cosechada']/row['area'] *100, axis=1)
    
    
    list_obj_soca = []
    # recorres el dataframe de avance por soca para convertir en obj cada fila
    for index, row in avance_corte.iterrows():
        soca = {
            'soca':row['soca'],
            'area_cosecha':row['area_cosechada'],
            'area':row['area'],
            'porcen_avan':row['porcen_avan']
        }
        list_obj_soca.append(collections.namedtuple("soca", soca.keys())(*soca.values()))
    data_obj_props['socas'] = list_obj_soca
    list_obj_props_soca.append(collections.namedtuple("propiedad", data_obj_props.keys())(*data_obj_props.values()))
list_obj_props_soca

[propiedad(prop='LA_ESTACADA--AGUILERA', socas=[soca(soca=0.0, area_cosecha=62.3, area=144.88, porcen_avan=43.00110436223081), soca(soca=1.0, area_cosecha=46.410000000000004, area=167.7, porcen_avan=27.67441860465117), soca(soca=2.0, area_cosecha=0.0, area=103.16, porcen_avan=0.0), soca(soca=3.0, area_cosecha=0.0, area=81.21, porcen_avan=0.0), soca(soca=4.0, area_cosecha=0.0, area=107.61, porcen_avan=0.0), soca(soca=5.0, area_cosecha=16.98, area=50.66, porcen_avan=33.51756810106593), soca(soca=6.0, area_cosecha=0.0, area=46.769999999999996, porcen_avan=0.0), soca(soca=7.0, area_cosecha=6.17, area=33.71, porcen_avan=18.303174132304953), soca(soca=8.0, area_cosecha=0.0, area=14.81, porcen_avan=0.0), soca(soca=9.0, area_cosecha=0.0, area=65.66, porcen_avan=0.0), soca(soca=11.0, area_cosecha=0.0, area=2.89, porcen_avan=0.0), soca(soca=13.0, area_cosecha=22.07, area=37.98, porcen_avan=58.10953133228015)]),
 propiedad(prop='FLAMBOYAN--AGUILERA', socas=[soca(soca=0.0, area_cosecha=62.3, area=

## TABLA AVANCE GENERAL

In [161]:
fecha_anterior = datetime(2023, 5, 5)
fecha_actual = datetime.now()

In [162]:
dias_zafra = (fecha_actual - fecha_anterior).days

In [166]:
totales_avance_gral = {
    'avance_area_porcen' : propiedades_totales['total_avance_porcen'],
    'avance_tn_porcen' : grupo_totales['total_avance_porcen'],
    'tiempo_porcen' : (dias_zafra / 171) * 100,
    'area_requerida' : propiedades_totales['total_area'] * (dias_zafra / 171),
    'cumplimiento_area_porcen' : propiedades_totales['total_area_cosechada']/(propiedades_totales['total_area'] * (dias_zafra / 171)) * 100,
    'tn_requeridas' : (propiedades_totales['total_proudccion'])*(dias_zafra / 171),
    'cumplimiento_tn_porcen' : grupo_totales['total_entrega']/((propiedades_totales['total_proudccion'])*(dias_zafra / 171)) * 100
}
totales_avance_gral
obj_avance_gral = collections.namedtuple("avance_gral", totales_avance_gral.keys())(*totales_avance_gral.values())
obj_avance_gral

avance_gral(avance_area_porcen=17.96065527863344, avance_tn_porcen=24.232104892529495, tiempo_porcen=52.046783625730995, area_requerida=446.0617543859649, cumplimiento_area_porcen=34.508674748835034, tn_requeridas=32872.97173180449, cumplimiento_tn_porcen=48.27798998362363)

# Convertir a PDF

In [249]:
import os

In [250]:
from docx2pdf import convert

In [251]:
path = r'reportes/'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 1 INFORMES
1 INFORMES


['reportes//911_ADC_09-06-2023_BALCAZAR JUSTINIANO MARIO.docx']

In [252]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdfs'))
    pdf = filename.replace('.docx','.pdf').split('/')[-1]
    convert(filename, r'pdf/' + pdf)
    i+=1
print('END')

CONVIRTIENDO... INFORME 1 reportes//911_ADC_09-06-2023_BALCAZAR JUSTINIANO MARIO.docx
reportes//911_ADC_09-06-2023_BALCAZAR JUSTINIANO MARIO.docx
reportes//911_ADC_09-06-2023_BALCAZAR JUSTINIANO MARIO.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

END


# leer ruta de planos

In [253]:
path = r'pdf/'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')

SE ENCONTRARON 1 INFORMES
1 INFORMES


In [254]:
reporte = rutas[0]

In [255]:
reporte

'pdf//911_ADC_09-06-2023_BALCAZAR JUSTINIANO MARIO.pdf'

In [256]:
list_pdfs = []
list_pdfs.append(reporte)

In [257]:
list_pdfs

['pdf//911_ADC_09-06-2023_BALCAZAR JUSTINIANO MARIO.pdf']

In [258]:
path = r'D:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2023\ANALISIS DE CAÑA EN CAMPO 2023\REPORTE_AVANCE_COSECHA\PLANOS'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')

SE ENCONTRARON 1578 INFORMES
1578 INFORMES


In [259]:
data  = {}
for i in rutas:
    cod_prop = i.split('_')[-1].split('.')[0]
    data[cod_prop]=i

In [260]:
data['30']

'D:\\Ingenio Azucarero Guabira S.A\\UTEA - SEMANAL - AVANCE COSECHA\\2023\\ANALISIS DE CAÑA EN CAMPO 2023\\REPORTE_AVANCE_COSECHA\\PLANOS/PLANO_30.pdf'

In [261]:
df

,unidad_01,unidad_02,area,area_cosechada,TCH,sin_cosechar,produccion_total,produccion_restante,produccion_cosechado
0,732,EL_PUESTO--BALCAZAR,18.59,1.22,46.944692,17.37,872.701825,815.429300,57.272524
1,733,LAS_PARCELAS--BALCAZAR,20.18,0.00,45.217658,20.18,912.492343,912.492343,0.000000
2,749,SAN_JORGE_DE_NAICO(EL_TRILLO)--BALCAZAR,25.09,4.28,43.599069,20.81,1093.900635,907.296621,186.604014
3,750,EL_CARMEN_NAICO--BALCAZAR,7.06,0.00,47.181886,7.06,333.104118,333.104118,0.000000
4,1694,EL_TRILLO--BALCAZAR_RAMON,25.88,1.24,48.479291,24.64,1254.644064,1194.529742,60.114321
5,1818,EL_PUESTO--BALCAZAR_RAMON,8.10,4.04,45.167463,4.06,365.856452,183.379901,182.476551
6,1819,EL_PUESTO--BALCAZAR_SOLETO_MARIO,3.54,0.00,39.509156,3.54,139.862413,139.862413,0.000000
7,1820,SAN_JORGE--BALCAZAR_SOLETO_MARIO,4.71,0.00,53.842054,4.71,253.596073,253.596073,0.000000


In [262]:
for row in df.itertuples():
    cod_prop = row.unidad_01
    ruta_prop = data[str(cod_prop)]
    list_pdfs.append(ruta_prop)

In [263]:
list_pdfs

['pdf//911_ADC_09-06-2023_BALCAZAR JUSTINIANO MARIO.pdf',
 'D:\\Ingenio Azucarero Guabira S.A\\UTEA - SEMANAL - AVANCE COSECHA\\2023\\ANALISIS DE CAÑA EN CAMPO 2023\\REPORTE_AVANCE_COSECHA\\PLANOS/PLANO_732.pdf',
 'D:\\Ingenio Azucarero Guabira S.A\\UTEA - SEMANAL - AVANCE COSECHA\\2023\\ANALISIS DE CAÑA EN CAMPO 2023\\REPORTE_AVANCE_COSECHA\\PLANOS/PLANO_733.pdf',
 'D:\\Ingenio Azucarero Guabira S.A\\UTEA - SEMANAL - AVANCE COSECHA\\2023\\ANALISIS DE CAÑA EN CAMPO 2023\\REPORTE_AVANCE_COSECHA\\PLANOS/PLANO_749.pdf',
 'D:\\Ingenio Azucarero Guabira S.A\\UTEA - SEMANAL - AVANCE COSECHA\\2023\\ANALISIS DE CAÑA EN CAMPO 2023\\REPORTE_AVANCE_COSECHA\\PLANOS/PLANO_750.pdf',
 'D:\\Ingenio Azucarero Guabira S.A\\UTEA - SEMANAL - AVANCE COSECHA\\2023\\ANALISIS DE CAÑA EN CAMPO 2023\\REPORTE_AVANCE_COSECHA\\PLANOS/PLANO_1694.pdf',
 'D:\\Ingenio Azucarero Guabira S.A\\UTEA - SEMANAL - AVANCE COSECHA\\2023\\ANALISIS DE CAÑA EN CAMPO 2023\\REPORTE_AVANCE_COSECHA\\PLANOS/PLANO_1818.pdf',
 'D:\\Inge

In [264]:
from PyPDF2 import PdfMerger

In [265]:
filename.replace('reportes', 'pdf_final').replace('docx', 'pdf')

'pdf_final//911_ADC_09-06-2023_BALCAZAR JUSTINIANO MARIO.pdf'

In [266]:
nombre_archivo_salida = filename.replace('reportes', 'pdf_final').replace('docx', 'pdf')

In [267]:
fusionador = PdfMerger()

In [268]:
for pdf in list_pdfs:
    fusionador.append(open(pdf, 'rb'))

In [269]:
with open(nombre_archivo_salida, 'wb') as salida:
    fusionador.write(salida)